In [ ]:
from src.custom_agents.planner_agent import PlannerAgent

planner_agent = PlannerAgent()


In [ ]:
await planner_agent.execute("Make a SWOT analysis for apple Inc.")

In [ ]:
from src.custom_agents.base import BaseAgent
import os 
agent = BaseAgent("test bot", "", os.getenv(""))
await agent.execute("Make a SWOT analysis for apple Inc.")

In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI, AsyncAzureOpenAI

_ = load_dotenv()

client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

In [ ]:
from agents import Agent, Runner, OpenAIChatCompletionsModel

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant",
    model=OpenAIChatCompletionsModel(
        model=os.getenv("GPT4O_MINI_DEPLOYMENT"),
        openai_client=client,
    )
)

In [ ]:
result = await Runner.run(
    agent,
    "Make a SWOT analysis for apple Inc.",
)

In [ ]:
result.raw_responses

In [ ]:
from custom_agents.swot_agent import SWOTAgent

planner_agent = SWOTAgent()
resutl = await planner_agent.execute("Make a SWOT analysis for apple Inc.")

In [ ]:
for output in resutl.final_output:
    print(output)

In [ ]:
from custom_agents.base_agent import BaseAgent

base_agent = BaseAgent(
    name="Chatbot",
)

In [ ]:
result = await base_agent.execute("Wie alt wurde Albert Einstein am 14. März 1879?")

In [ ]:
result.final_output

In [ ]:
# from dotenv import load_dotenv
# import os
from manager import SWOTAnalysisManager

# load_dotenv()

# print(os.getenv("O3_MINI_DEPLOYMENT"))

swot_manager = SWOTAnalysisManager()

In [ ]:
result = await swot_manager.run("Make a very detailed SWOT analysis for SAP and use data from 2025.")

# Baue den SWOTAnalysisManager

## Aufbau
1. Planner
2. Searcher
3. SWOT

### 1. Planner

In [ ]:
from agents import Runner

query = "Make a very detailed SWOT analysis for SAP and use data from 2025."

In [ ]:
from custom_agents.planner_agent import PlannerAgent, WebSearchPlan, WebSearchItem
from openai.types.responses import ResponseTextDeltaEvent

planner_agent = PlannerAgent()
planner_result = planner_agent.execute_streamed(f"Query: {query}")

In [ ]:
for plan in planner_result.final_output_as(WebSearchPlan).searches:
    print(f"{plan.query} ----> {plan.reason}")

### 2. Search

In [ ]:
from custom_agents.search_agent import SearchAgent

search_agent = SearchAgent()

In [ ]:
async def search(item: WebSearchItem):
    input_data = f"Search term: {item.query}\nReason: {item.reason}\n"
    result = await search_agent.execute(input_data)
    return result.final_output
    

In [ ]:
import asyncio
from agents import custom_span

tasks = [asyncio.create_task(search(item)) for item in planner_result.final_output_as(WebSearchPlan).searches]

with custom_span("Search for all items"):
    # await asyncio.gather(*tasks)
    search_resutls = []
    for task in asyncio.as_completed(tasks):
        result = await task
        search_resutls.append(result)

In [ ]:
search_resutls

### 3. SWOT Analysis

In [ ]:
from custom_agents.swot_agent import SWOTAgent

swot_agent = SWOTAgent()

input_data = f"Original query: {query}\nSummarized search results: {search_resutls}"

swot_result = await swot_agent.execute(input_data, max_turns=20)

In [ ]:
swot_result.final_output.swot_summary

### 4. Test New Manager Class

In [ ]:
from manager import SWOTAnalysisManager

query = "Make a very detailed SWOT analysis for SAP and use data from 2025."

swot_manager = SWOTAnalysisManager()
swot_result = await swot_manager.execute_streamed(query)

In [ ]:
for event in swot_result.stream_events():
    if isinstance(event, ResponseTextDeltaEvent):
        print(event.delta.content, end="")
    else:
        print(event)

# New Idea
## Aufbau
1. Planner ohne Stream
2. Search ohne Stream
3. SWOT mit Stream

In [ ]:
from manager import SWOTAnalysisManager

agent = SWOTAnalysisManager()

query = "Make a SWOT analysis for Deutsche Telekom"

In [ ]:
planner_result = await agent.planner_agent.execute(f"Query: {query}")

In [ ]:
n = 1
planner_answer = ""
for item in planner_result.final_output.searches:
    planner_answer += f"{n}. Search Query: {item.query} <--> Reason {item.reason}\n"
    n += 1

In [ ]:
planner_answer 

# 10k-Filings Retrieval

In [ ]:
# 1. Import the necessary functions from edgartools
from edgar import *

# 2. Tell the SEC who you are
set_identity("mike@indigo.com")

# 3. Start using the library
filings = get_filings()

In [ ]:
company = Company("AAPL") # or Company("0000320193") or Company(320193)

In [ ]:
filings = company.get_filings(form='10-K').latest()

In [ ]:
filings.attachments[1].download("./")

In [ ]:
filing = filings.attachments[1]

In [ ]:
type(filing)

In [ ]:
type(filings)